In [2]:
## Arxiv research paper
## Tools creation

from langchain_community.tools import ArxivQueryRun,WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper,ArxivAPIWrapper

In [3]:
## Used the inbuilt tool of wikipedia
api_wrapper_wiki=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=250)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper_wiki)
wiki.invoke("what is the name of indian prime minister")

'Page: List of prime ministers of India\nSummary: The prime minister of India is the chief executive of the Government of India and chair of the Union Council of Ministers. Although the president of India is the constitutional, nominal, and ceremonial '

In [4]:
api_wrapper_arxiv=ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=250)
arxiv=ArxivQueryRun(api_wrapper=api_wrapper_arxiv)
arxiv.name

'arxiv'

In [5]:
tools=[wiki,arxiv]


In [6]:
## Making custom tools
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
import os
from dotenv import load_dotenv
load_dotenv()



embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
os.environ['HUGGINGFACEHUB_API_TOKEN']=os.getenv("HUGGINGFACEHUB_API_TOKEN")

USER_AGENT environment variable not set, consider setting it to identify your requests.
c:\Users\ritik\anaconda3\envs\search\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:
loader=WebBaseLoader("https://docs.smith.langchain.com/")
docs=loader.load()
splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=250)
chunks=splitter.split_documents(docs)
vectordb=FAISS.from_documents(chunks,embeddings)
retriever=vectordb.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000206E34614B0>, search_kwargs={})

## CONVERTING THE TOOL INTO A RETRIEVER

In [17]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool=create_retriever_tool(retriever,name="Lagmsith-search",description="Search any information about Langsmith")
retriever_tool.description

'Search any information about Langsmith'

In [19]:
tools=[wiki,arxiv,retriever_tool]

In [22]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\ritik\\anaconda3\\envs\\search\\lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250)),
 Tool(name='Lagmsith-search', description='Search any information about Langsmith', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x00000206DE145E10>, retriever=VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000206E34